In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
from keras.models import Sequential
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.core import Dense, Dropout
from imutils import paths
import cv2
import os
import numpy as np
import sklearn
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from keras.applications.resnet import ResNet50
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from keras.regularizers import l2

In [10]:
#Defining various classes
classes=['Destroyed','Major_Damage','Minor_Damage','No_Damage']

DATASET_PATH = "Data"

#Spilting dataset
Train_Split=0.75
Val_Split=0.1
Test_Split=0.25

imagePaths = list(paths.list_images(DATASET_PATH))
# print(imagePaths)
data = []
labels = []

#Processing Images
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[1]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    data.append(image)
    labels.append(label)

print(labels)

['Destroyed', 'Destroyed', 'Destroyed', 'Destroyed', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Major_Damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Minor_damage', 'Mi

In [11]:
#Processing Data
data = np.array(data)/255
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=Test_Split, random_state=42,stratify=labels )
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)
# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
    test_size=Val_Split, random_state=84,stratify=trainY)
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)

(471, 200, 200, 3)
(157, 200, 200, 3)
(471, 4)
(157, 4)
(423, 200, 200, 3)
(423, 4)
(48, 200, 200, 3)
(48, 4)


In [12]:
import pandas as pd
def counter(trainY):
  dt=pd.DataFrame(trainY)
  return dt.value_counts()
print(counter(trainY))

0  1  2  3
0  0  0  1    338
      1  0     70
   1  0  0     12
1  0  0  0      3
dtype: int64


In [31]:
def predict_(model,x_test,y_test):
    y_predict = np.asarray(model.predict(x_test))
    print(y_predict.shape)
    y_predict =y_predict.reshape(3,628).argmax(axis=0) 
    acc=sklearn.metrics.accuracy_score(y_test, y_predict)
    score=sklearn.metrics.f1_score(y_test, y_predict,average='micro')
    print("The micro F1 score of the model is", score)
    print("The accuracy score of the model is", acc)

def recall_m(y_true, y_pred):
        true_positives =int( K.sum(K.round(K.clip(y_true * y_pred, 0, 1))))
        possible_positives = int(K.sum(K.round(K.clip(y_true, 0, 1))))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives =int( K.sum(K.round(K.clip(y_true * y_pred, 0, 1))))
        predicted_positives =int( K.sum(K.round(K.clip(y_pred, 0, 1))))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
 rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True,
 fill_mode='nearest')
img_iter = datagen.flow(trainX, trainY, batch_size=64)

In [ ]:
def run_model(model,es_round=5,epoch=10,batch_size=64):
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
    cb=EarlyStopping(monitor='val_accuracy', min_delta=0, patience=es_round, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.fit(img_iter,batch_size=64,validation_data=(valX,valY),callbacks=[cb],epochs=epoch,verbose=1)
    score=model.evaluate(testX,testY)
    print('Evaluating Test Score..........')
    print("Test Loss:{} ".format(round(score[0],2)))
    print("Test Accuracy:{} %".format(int(round(score[1]*100))))
    print('completed.')
    

In [ ]:
def generate_xBD_baseline_model():
  weights = 'imagenet'
  inputs = Input(shape=(200, 200, 3))

  base_model = ResNet50(include_top=False, weights=weights, input_shape=(200, 200, 3))

  for layer in base_model.layers:
    layer.trainable = False

  x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(200, 200, 3))(inputs)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)
  x = Dropout(0.6)(x)

  
  x = Flatten()(x)

  base_resnet = base_model(inputs)
  base_resnet = Flatten()(base_resnet)

  concated_layers = Concatenate()([x, base_resnet])

  concated_layers = Dense(124, activation='relu' )(concated_layers)
  concated_layers = Dense(64, activation='relu' )(concated_layers)
  output = Dense(4, activation='softmax')(concated_layers)

  model = Model(inputs=inputs, outputs=output)
  return model

In [ ]:
model=generate_xBD_baseline_model() 
run_model(model)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
7/7 [==============================] - 87s 12s/step - loss: 7.6438 - accuracy: 0.6194 - val_loss: 12.3080 - val_accuracy: 0.7917
Epoch 2/10
7/7 [==============================] - 82s 12s/step - loss: 7.9839 - accuracy: 0.6738 - val_loss: 1.0659 - val_accuracy: 0.7917
Epoch 3/10
7/7 [==============================] - 82s 12s/step - loss: 3.5701 - accuracy: 0.6265 - val_loss: 3.2358 - val_accuracy: 0.7917
Epoch 4/10
7/7 [==============================] - 81s 12s/step - loss: 2.4973 - accuracy: 0.5768 - val_loss: 2.0441 - val_accuracy: 0.7917
Epoch 5/10
7/7 [==============================] - 82s 12s/step - loss: 1.7795 - accuracy: 0.5957 - val_loss: 1.6070 - val_accuracy: 0.7917
Epoch 6/10
5/5 [==============================] - 22s 4s/step - loss: 12.5662 - accuracy: 0.8025
Evaluating Test Score..........
Test Loss:12.57 
Test Accuracy:80 %
completed.


In [ ]:
model.save("Damages.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [6]:
m=keras.models.load_model("Damages.h5")
m.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 200, 200, 32  2432        ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 max_pooling2d_12 (MaxPooling2D  (None, 100, 100, 32  0          ['conv2d_12[0][0]']              
 )                              )                                                           

In [11]:
m.evaluate(testX,testY)

5/5 [==============================] - 23s 4s/step - loss: 12.5662 - accuracy: 0.8025


[12.566165924072266, 0.8025477528572083]

In [30]:
y_pred = m.predict(testX, batch_size=64, verbose=0)
print(recall_m(testY, y_pred))

0.80254777018946


In [32]:
print(precision_m(testY, y_pred))

0.80254777018946


In [33]:
print(f1_m(testY, y_pred))

0.8025477201894631
